In [1]:
import os
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from keras.models import load_model


tf.device('/gpu:1')
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = "true"

import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 1
session = tf.compat.v1.Session(config=config)

import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

import PreProcessing
import net
import PatchExtractor

from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

Physical devices cannot be modified after being initialized
GPU selected: 0
Physical devices cannot be modified after being initialized


In [2]:
in_path = 'giuriati_2/20170621_deg0_HHVV.npy'
out_path = 'cnn_article'
architecture = 'Auto3D2'
ny = 3 ##3
data_augmentation = True
preprocessing = 'normalize'
patch_size = 128  ##64
patch_stride = 8 ##4
n_bsc = 2 #5 

In [3]:
def parse_pp(string):
    return getattr(PreProcessing, string)

def parse_net(string):
    return getattr(net, string)

In [4]:
if not os.path.exists(out_path):
    os.makedirs(out_path)

field, campaign = in_path.split('/')
campaign, extension = campaign.split('.')

In [5]:
dataset = np.load('./datasets/'+str(in_path),allow_pickle=True).item()

train_bsc_idx = np.where(np.asarray(dataset['ground_truth']) == 0)[0][:n_bsc]
trainset = dataset['data'][train_bsc_idx]
trainset = np.moveaxis(trainset, np.argmin(trainset.shape), -1)
del dataset


if patch_size is not None:
    patch_size = (patch_size, patch_size)
else:
    patch_size = trainset.shape[1:]

patch_size = patch_size + (ny,)
patch_stride = (patch_stride, patch_stride, 1)

In [6]:
from sklearn.metrics import roc_curve, roc_auc_score
from tqdm import tqdm
training = './cnn_article/giuriati_2_20170621_deg0_HHVV_Auto3D2_patch64_stride4_bsc5_ny3'
net_weights = training + '.h5'


In [7]:
train_path = in_path
dataset = np.load('./datasets/' + str(in_path),allow_pickle=True).item()

data = dataset['data']

pe = PatchExtractor.PatchExtractor(patch_size, stride=patch_stride)

gt = np.asarray(dataset['ground_truth'])
del dataset

test_idx = np.arange(data.shape[0])

if in_path == train_path:
    train_idx = np.where(gt == 0)[0][:n_bsc]
    test_idx = np.delete(test_idx, train_idx)
testset = data[test_idx]
gt = gt[test_idx]
del data
testset = np.moveaxis(testset, np.argmin(testset.shape), -1)
ytest=gt
xt=testset

In [8]:
autoencoder, encoder = parse_net(architecture)(patch_size)
autoencoder.load_weights(os.path.join(net_weights))

out_name = field+'_'+campaign+'_'+architecture+'_patch'+str(patch_size[0])+'_stride'+str(patch_stride[0])+'_bsc'+str(n_bsc)+'_ny'+str(ny)

In [9]:
model = load_model('model.h5')

In [10]:
#######################################################
#######################################################
#######################################################
restuv=[]
cnnuv=[]
threads=[]
import threading
import time 

start_time = time.time()
    
def funloo(y,gh):
    testset=xt[:,:,y*3:(y*3)+3]       ###3
    yte=ytest[y*3:(y*3)+3]
    xtest12=testset.reshape(3,170,440,1)
    
    patches = pe.extract(testset)
    cnnuv.append(list(model.predict(xtest12)))
    cnnuv.append(list(yte))
    del xtest12
    del testset
    patchesIdx = patches.shape
    patches_hat = autoencoder.predict(patches.reshape((-1,) + patch_size))
    mseFeat = (encoder.predict(patches.reshape((-1,) + patch_size)) - encoder.predict(patches_hat))**2
    del patches
    uv10001=patches_hat.shape
    del patches_hat
    mseFeat_patches = np.zeros(uv10001) + np.mean(mseFeat, axis=(1,2,3)).reshape((-1,1,1,1))

    del mseFeat
    mseFeat_vol = pe.reconstruct(mseFeat_patches.reshape(patchesIdx))
    del mseFeat_patches
    mse_mask_max = np.max(mseFeat_vol, axis=(0, 1))
    #print(mse_mask_max)
    restuv.append(list(mse_mask_max))
    restuv.append(yte)
    print("ok")
    
    

for i in range(20):
    try:
        th = threading.Thread(target=funloo,args=(i,i))
        th.start()
        threads.append(th)
    except:
        print("error")
        pass
for th in threads:
    th.join()
    
end_time = time.time()
timetaken=(end_time-start_time)

ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
ok
okok



In [32]:
restuv=np.array(restuv)
cnnuv=np.array(cnnuv)

In [33]:
restuv.shape,cnnuv.shape

((40,), (40,))

In [17]:
cnnuv=cnnuv[:,1]
ydt2=[]
yt2=[]
for i in range(cnnuv.shape[0]):
    if(i%2==0):
        ydt2.append(cnnuv[i])
    else:
        yt2.append(cnnuv[i])
restuv=restuv[:,1]
ydt1=[]
yt1=[]
for i in range(restuv.shape[0]):
    if(i%2==0):
        ydt1.append(restuv[i])
    else:
        yt1.append(restuv[i])

In [21]:
timetaken

60.5344500541687

In [30]:
fpr_max, tpr_max, thresholds_max = roc_curve(yt1, ydt1)
roc_auc_max = roc_auc_score(yt1, ydt1)
print('best AUC = %0.2f' % roc_auc_max)

best AUC = 0.86
